In [1]:
import torch

# Simplified Self Attention Mechanism

In [2]:
inputs = torch.tensor(
[[0.43, 0.15, 0.89], # Your
[0.55, 0.87, 0.66], # journey
[0.57, 0.85, 0.64], # starts
[0.22, 0.58, 0.33], # with
[0.77, 0.25, 0.10], # one
[0.05, 0.80, 0.55]] # step
)

# Attention vecto for Single Query

In [3]:
query = inputs[1]
print(inputs.shape[0])
attn_scores_2 = torch.empty(inputs.shape[0])
for i, x_i in enumerate(inputs):
    attn_scores_2[i] = torch.dot(x_i, query)
print(attn_scores_2)

6
tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])


In [4]:
def calculate_attn_scores(query, inputs):
    attn_scores = torch.empty(inputs.shape[0])
    for i, k_i in enumerate(inputs):
        print(f'query: {query.shape} and key {k_i.shape}')
        attn_scores[i] = torch.dot(k_i, query)
        # normalization
    # attn_scores = attn_scores / attn_scores.sum()
    # attn_scores = softmax(attn_scores)
    attn_scores = torch.softmax(attn_scores, dim=0)
    return attn_scores


def softmax(x):
    return torch.exp(x) / torch.exp(x).sum()

In [5]:
attn_weights_v2 = calculate_attn_scores(query, inputs)

query: torch.Size([3]) and key torch.Size([3])
query: torch.Size([3]) and key torch.Size([3])
query: torch.Size([3]) and key torch.Size([3])
query: torch.Size([3]) and key torch.Size([3])
query: torch.Size([3]) and key torch.Size([3])
query: torch.Size([3]) and key torch.Size([3])


In [6]:
query = inputs[1]
context_vec_2 = torch.zeros(query.shape)
for i, x_i in enumerate(inputs):
    context_vec_2 += attn_weights_v2[i]*x_i

In [7]:
print(context_vec_2)

tensor([0.4419, 0.6515, 0.5683])


# For all Queries, Keys and Values 

# context_matrix = softmax(inputs*inputs.T)

In [8]:
attn_scores = torch.empty(inputs.shape[0], inputs.shape[0])
for i, query in enumerate(inputs):
    for j, key in enumerate(inputs):
        attn_scores[i,j] = torch.dot(query, key)
context_matrix = torch.softmax(attn_scores, dim=-1)
print(context_matrix)

tensor([[0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452],
        [0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581],
        [0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.1263, 0.1720],
        [0.1526, 0.1958, 0.1975, 0.1367, 0.1879, 0.1295],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])


In [9]:
attn_scores = inputs @ inputs.T

In [10]:
context_matrix = torch.softmax(attn_scores, dim=-1)
print(context_matrix)

tensor([[0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452],
        [0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581],
        [0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.1263, 0.1720],
        [0.1526, 0.1958, 0.1975, 0.1367, 0.1879, 0.1295],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])


# Context_matrix * Inputs

In [11]:
out = context_matrix @ inputs

In [12]:
print(out)

tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])


# Self Attention Mechanism

In [13]:
x_2 = inputs[1]
d_in = inputs.shape[1]
d_out = 2

In [14]:
torch.manual_seed(123)
W_query = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=True)
W_key = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=True)
W_value = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=True)


In [15]:
query = x_2 @ W_query
key = x_2 @ W_key
value = x_2 @ W_value

In [16]:
print(query)

tensor([0.4306, 1.4551], grad_fn=<SqueezeBackward3>)


In [17]:
query = inputs @ W_query
key = inputs @ W_key
value = inputs @ W_value

In [18]:
print(f'query shape: {query.shape}, key shape: {key.shape}, value: {value.shape}')

query shape: torch.Size([6, 2]), key shape: torch.Size([6, 2]), value: torch.Size([6, 2])


In [19]:
attention_score = query @ key.T
d_k = key.shape[-1]
atten_weights = torch.softmax(attention_score/ d_k **5, dim=-1)

In [20]:
context_vector = atten_weights @ value

# Self Attention Python Class

In [21]:
import  torch.nn as nn

class SelfAttention_v1(nn.Module):
    def __init__(self, d_in, d_out):
        super().__init__()
        self.W_q = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=True)
        self.W_k = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=True)
        self.W_v = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=True)

    def forward(self , x):
        Q = x @ self.W_q
        K = x @ self.W_k
        V = x @ self.W_v
        d_k = K.shape[-1]
        attn_score = Q @ K.T
        
        attn_score_scaled = attn_score/d_k**0.5
        attn_weights = torch.softmax(attn_score_scaled, dim=-1)

        context_vec = attn_weights @ V
        return context_vec
        

In [22]:
import  torch.nn as nn

class SelfAttention_v2(nn.Module):
    def __init__(self, d_in, d_out, qkv_bias=False):
        super().__init__()
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key   =  nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)

    def forward(self , x):
        Q = self.W_query(x)
        K = self.W_key(x)
        V = self.W_value(x)
        d_k = K.shape[-1]
        attn_score = Q @ K.T
        
        attn_score_scaled = attn_score / d_k**0.5
        attn_weights = torch.softmax(attn_score_scaled, dim=-1)

        context_vec = attn_weights @ V
        return context_vec

In [23]:
selfattn = SelfAttention_v2(3, 2)

In [24]:
# x = selfattn(inputs)
# print(inputs)
# print(x)

# Mask Attention

In [25]:
import  torch.nn as nn

class SelfAttention_v3(nn.Module):
    def __init__(self, d_in, d_out, qkv_bias=False):
        super().__init__()
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key   =  nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)

    def forward(self , x):
        Q = self.W_query(x)
        K = self.W_key(x)
        V = self.W_value(x)
        d_k = K.shape[-1]
        attn_score = Q @ K.T
        mask = torch.tril(torch.ones(x.shape[0], x.shape[0]))
        
        attn_score_scaled = attn_score / d_k**0.5
        masked_attn_weights = torch.softmax(attn_score_scaled, dim=-1) * mask
        masked_attn_weights = masked_attn_weights / masked_attn_weights.sum(dim=-1, keepdim=True)
        print(masked_attn_weights)
        context_vec = masked_attn_weights @ V
        return context_vec

In [26]:
selfattn = SelfAttention_v3(3, 2)

In [27]:
x = selfattn(inputs)
print(x)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5437, 0.4563, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3728, 0.3128, 0.3144, 0.0000, 0.0000, 0.0000],
        [0.2642, 0.2425, 0.2431, 0.2502, 0.0000, 0.0000],
        [0.2146, 0.1890, 0.1896, 0.1969, 0.2099, 0.0000],
        [0.1760, 0.1605, 0.1610, 0.1664, 0.1749, 0.1612]],
       grad_fn=<DivBackward0>)
tensor([[0.4566, 0.2729],
        [0.5792, 0.3011],
        [0.6249, 0.3102],
        [0.5691, 0.2785],
        [0.5543, 0.2520],
        [0.5337, 0.2499]], grad_fn=<MmBackward0>)


In [28]:
import  torch.nn as nn

class SelfAttention_v4(nn.Module):
    def __init__(self, d_in, d_out, qkv_bias=False):
        super().__init__()
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key   =  nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)

    def forward(self , x):
        Q = self.W_query(x)
        K = self.W_key(x)
        V = self.W_value(x)
        d_k = K.shape[-1]
        
        attn_score = Q @ K.T
        mask = torch.triu(torch.ones(x.shape[0], x.shape[0]), diagonal=1)
        attn_score_scaled = attn_score / d_k**0.5
        masked_attn_weights = attn_score_scaled.masked_fill(mask.bool(), -torch.inf)

        masked_attn_weights = torch.softmax(masked_attn_weights, dim=-1)
        context_vec = masked_attn_weights @ V
        return context_vec

In [29]:
selfattn = SelfAttention_v4(3,2)
x = selfattn(inputs)

In [30]:
print(x)

tensor([[-0.5684,  0.5063],
        [-0.5388,  0.6447],
        [-0.5242,  0.6954],
        [-0.4578,  0.6471],
        [-0.4006,  0.5921],
        [-0.3997,  0.5971]], grad_fn=<MmBackward0>)


# Implementing a compact causal attention class

In [82]:
class CausalAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout ,qkv_bias=False):
        super().__init__()
        self.d_out = d_out
        
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.dropout = nn.Dropout(0.5)
        self.register_buffer(
        'mask',
        torch.triu(torch.ones(context_length, context_length), diagonal=1)
        )
        
    
    def forward(self, x):
        
        b, num_tokens, d_in = x.shape
        Q = self.W_query(x)
        K = self.W_key(x)
        V = self.W_value(x)
        attn_score = Q @ K.transpose(1,2)
        attn_scores.masked_fill_(
        self.mask.bool()[:num_tokens, :num_tokens], -torch.inf)
        
        attn_weights = torch.softmax(
            attn_scores / self.d_out**0.5, dim=-1)
        
        attn_weights = self.dropout(attn_weights)
        
        context_vec = attn_weights @ V
        return context_vec

In [83]:
ca = CausalAttention(3,2,6,0.5)

In [88]:
batch = torch.stack((inputs, inputs, inputs), dim=0)
x = ca(batch)

In [89]:
print(x)

tensor([[[-1.1360, -0.1767],
         [-1.5790, -0.3089],
         [-0.9782, -0.1781],
         [ 0.0000,  0.0000],
         [-0.6150, -0.1111],
         [-0.7535, -0.1529]],

        [[-1.1360, -0.1767],
         [-1.5790, -0.3089],
         [-0.9782, -0.1781],
         [ 0.0000,  0.0000],
         [-0.6150, -0.1111],
         [-0.7535, -0.1529]],

        [[-1.1360, -0.1767],
         [-1.5790, -0.3089],
         [-0.9782, -0.1781],
         [ 0.0000,  0.0000],
         [-0.6150, -0.1111],
         [-0.7535, -0.1529]]], grad_fn=<CloneBackward0>)


In [90]:
class MultiHeadAttentionWrapper(nn.Module):
    def __init__(self, d_in, d_out, context_length , 
                 dropout,num_heads, qkv_bias=False):
        super().__init__()
        self.heads = [CausalAttention(
            d_in, d_out, 
            context_length, dropout)
            for _ in  range(num_heads)]
        
    def forward(self, x):
        for head in self.heads:
            return torch.cat([head(x) for head in self.heads], dim=-1)

In [100]:
multiheadattentionwarpper = MultiHeadAttentionWrapper(3,2,6,0.5,3)
out = multiheadattentionwarpper(batch)

In [101]:
lp = nn.Linear(6,2)
out = lp(out)

In [102]:
print(out.shape)

torch.Size([3, 6, 2])


In [148]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, num_heads,dropout, qkv_bias=False):
        super().__init__()
        assert(d_out%num_heads==0), \
        'd_out must be divisible by num_heads'
        
        self.d_in = d_in
        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = self.d_out // self.num_heads
        
        self.W_Q = nn.Linear(self.d_in, self.d_out, qkv_bias)
        self.W_K = nn.Linear(self.d_in, self.d_out, qkv_bias)
        self.W_V = nn.Linear(self.d_in, self.d_out, qkv_bias)
        
        self.register_buffer(
        'mask',
        torch.triu(torch.ones(context_length, context_length), diagonal=1)
        )
        self.projection = nn.Linear(self.d_out, self.d_out)
        self.dropout = nn.Dropout(dropout)
        
        
    def forward(self, x):
        b, num_tokens, d_out = x.shape
        Q = self.W_Q(x)
        K = self.W_K(x)
        V = self.W_V(x)
        
        Q = Q.view(b, num_tokens, self.num_heads, self.head_dim)
        K = K.view(b, num_tokens, self.num_heads, self.head_dim)
        V = V.view(b, num_tokens, self.num_heads, self.head_dim)
        
        Q = Q.transpose(1,2)
        K = K.transpose(1,2)
        V = V.transpose(1,2)
        
        attn_score = Q @ K.transpose(2,3)
        
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]
        attn_scores.masked_fill_(mask_bool, -torch.inf)
        
        attn_weights = torch.softmax(
            attn_scores / K.shape[-1]**0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)
        context_vec = (attn_weights @ V).transpose(1,2)
        context_vec = context_vec.contiguous().view(
        b, num_tokens, self.d_out)
        out = self.projection(context_vec)
        return out

In [149]:
mha = MultiHeadAttention(3,2*5, 6 , 5, 0.5)
# mha.W_K(batch)
# mha.W_K(batch).view(3,6,5,2)
# (mha.W_Q(batch).view(3,6,5,2).transpose(1,2) @ mha.W_K(batch).view(3,6,5,2).transpose(1,2).transpose(2,3))

In [150]:
out = mha(batch)

torch.Size([3, 5, 6, 6])
torch.Size([3, 6, 10])


In [151]:
out.shape

torch.Size([3, 6, 10])

In [154]:
a = torch.tensor([[[[0.2745, 0.6584, 0.2775, 0.8573],
[0.8993, 0.0390, 0.9268, 0.7388],
[0.7179, 0.7058, 0.9156, 0.4340]],
[[0.0772, 0.3565, 0.1479, 0.5331],
[0.4066, 0.2318, 0.4545, 0.9737],
[0.4606, 0.5159, 0.4220, 0.5786]]]])
a.shape

torch.Size([1, 2, 3, 4])

In [153]:
a @ a.transpose(2,3)

tensor([[[[1.3208, 1.1631, 1.2879],
          [1.1631, 2.2150, 1.8424],
          [1.2879, 1.8424, 2.0402]],

         [[0.4391, 0.7003, 0.5903],
          [0.7003, 1.3737, 1.0620],
          [0.5903, 1.0620, 0.9912]]]])

In [157]:
first_head = a[0, 0, :, :]
first_res = first_head @ first_head.T
print("First head:\n", first_res)
second_head = a[0, 1, :, :]
second_res = second_head @ second_head.T
print("\nSecond head:\n", second_res)

First head:
 tensor([[1.3208, 1.1631, 1.2879],
        [1.1631, 2.2150, 1.8424],
        [1.2879, 1.8424, 2.0402]])

Second head:
 tensor([[0.4391, 0.7003, 0.5903],
        [0.7003, 1.3737, 1.0620],
        [0.5903, 1.0620, 0.9912]])
